# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [20]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [21]:
weather_df = pd.read_csv("output_data/clean_city_data.csv")
weather_df.head()

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
0,Kumul,42.80,93.45,79.11,30,0,10.80,CN,1595953854
1,Thompson,55.74,-97.86,60.80,87,90,5.82,CA,1595953746
2,Kalmar,56.66,16.36,69.80,77,77,5.82,SE,1595953858
3,Victoria Point,-27.58,153.30,55.00,62,0,4.00,AU,1595953858
4,Marawi,8.00,124.29,73.99,92,100,7.00,PH,1595953858


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [22]:
# configure gmaps with API key
gmaps.configure(api_key=g_key)

In [23]:
# store 'Lat' and 'Lng' into locations 
locations = weather_df[["Lat", "Lng"]].astype(float)
locations.head()

,Lat,Lng
0,42.80,93.45
1,55.74,-97.86
2,56.66,16.36
3,-27.58,153.30
4,8.00,124.29


In [24]:
# store 'Humidity' to float
humidity = weather_df["Humidity"].astype(float)
humidity.head()

0    30.0
1    87.0
2    77.0
3    62.0
4    92.0
Name: Humidity, dtype: float64

In [26]:
# create a humidity Heatmap layer

fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Remove rows missing data
best_cities_df = weather_df.dropna()

In [28]:
best_cities_df = best_cities_df[best_cities_df['Humidity']<=50]

In [29]:
best_cities_df = best_cities_df[best_cities_df['Max_Temp']<=75]

In [30]:
best_cities_df = best_cities_df[best_cities_df['Max_Temp']>=60]

In [31]:
best_cities_df = best_cities_df[best_cities_df['Wind_Speed']<=10]

In [32]:
best_cities_df = best_cities_df[best_cities_df['Cloudiness']<=25]

In [33]:
best_cities_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date
83,Kabompo,-13.59,24.20,65.16,35,0,5.12,ZM,1595953867
121,Karratha,-20.74,116.85,67.87,34,0,5.44,AU,1595953861
193,Solwezi,-12.17,26.39,62.26,39,0,4.52,ZM,1595953882
234,Kasane,-17.82,25.15,73.40,25,0,8.05,BW,1595953888
259,Presidencia Roque Sáenz Peña,-26.79,-60.44,60.26,33,22,10.00,AR,1595953891
262,East London,-33.02,27.91,67.78,50,0,1.79,ZA,1595953892
442,Graaff-Reinet,-32.25,24.53,66.99,27,0,1.01,ZA,1595953916
521,Shubarkuduk,49.14,56.49,72.01,40,10,8.25,KZ,1595953926


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [34]:
hotel_df = best_cities_df

In [35]:
hotel_df['Hotel_Name'] = ""
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
83,Kabompo,-13.59,24.20,65.16,35,0,5.12,ZM,1595953867,
121,Karratha,-20.74,116.85,67.87,34,0,5.44,AU,1595953861,
193,Solwezi,-12.17,26.39,62.26,39,0,4.52,ZM,1595953882,
234,Kasane,-17.82,25.15,73.40,25,0,8.05,BW,1595953888,
259,Presidencia Roque Sáenz Peña,-26.79,-60.44,60.26,33,22,10.00,AR,1595953891,
262,East London,-33.02,27.91,67.78,50,0,1.79,ZA,1595953892,
442,Graaff-Reinet,-32.25,24.53,66.99,27,0,1.01,ZA,1595953916,
521,Shubarkuduk,49.14,56.49,72.01,40,10,8.25,KZ,1595953926,


In [36]:
# find the closest hotel to each cities coordinates

params = {
    "types": "lodging",
    "radius": 5000,
    "key": g_key
}

# use the lat/lng to find hotels
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    response = requests.get(base_url, params=params)

    # print the name_address url, avoid doing for public github repos in order to avoid exposing key
    #print(response.url)

    # convert to json
    response = response.json()
    print(json.dumps(response, indent=4, sort_keys=True))

    # since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel_Name"] = response["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

{
    "html_attributions": [],
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -13.589597,
                    "lng": 24.1997071
                },
                "viewport": {
                    "northeast": {
                        "lat": -13.58833466970849,
                        "lng": 24.20054375
                    },
                    "southwest": {
                        "lat": -13.5910326302915,
                        "lng": 24.19719715
                    }
                }
            },
            "icon": "https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png",
            "id": "2e4a18c1c68c59e9833a9b318ac166449d81611a",
            "name": "Faroja City Iyindeko Cas-jay",
            "photos": [
                {
                    "height": 360,
                    "html_attributions": [
                        "<a href=\"https://maps.google.co

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAGW6sc0fgRP9nouOggslPlHvJlSyWESzzbUSRyVyhFtpTYpuFni6pigk_O1JNrdmVbFDmSDuHo9CETYHfp7-1qHgBWMwBFBnlsl8mnxaQ9HPiYal1KIIIPVG29V1PQXzMyExIYrq68GX3Rilj3VvHXEEejABeocfmxeJNxEB8yIOoFo9PLzRFOgWW305PjcpPxXJan6qF5GpyeE3ca_uuhDSniUdfFvlrCxdAUjDL1sivL5FF_GAGpfRPTlYYFjIIklN-UxKAnmbrLGO5vNDmq43bI6T2dp7S7oC9E3lI6mNGYnkQg_OV48JJRXBZ-a8EsY1wEGdi-Weor8oJF1nMq4-KW6eCKM05rkUwdjv12S3E6FhKornps4-cN0vgnDb0hIQLTVIPeA5NNeV7SWCmQmA7BoUgObSvlI1qqTl_TzjHxSWq4UzAHw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -20.7322255,
                    "lng": 116.8467438
                },
                "viewport": {
                    "northeast": {
                        "lat": -20.7308124197085,
                        "lng": 116.8481896302915
                    },
                    "southwest": {
                        "lat": -20.73351038029

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAPMaXrD-u_UAvBtGSfx068DBCdGYI7WB3KC0b-K0uvSjyHdKo40Toq3H9qU5DvOFocfe1nJj8QggM4YmAuMnmtGbjSjAvQabH1TIZzooP1XXTFmzCaIFrb7bKZHVqURmdkU9GUySp8FwdMz_MB-yxNlItpeJRh629Nv3Ndh1J8UFz44iMIWe22LoTYOcB-cDcZtdYNNFhHhfTLO_K24dmkRp9MQpfOSoi0ZUZoE-EzlDqMHCnQKD-ewfpjJJQQl2quzJHl1z4vY8PYtyUnKxpxYHrW6zPixo5zsMqgEYhr_Fyb4iLO-rnfbAEe9y6iPdE1R3hGdZkROpR8AgOVAgTq83STZsWhlciD-zrfB2BkoXs6_Rg0mLQ2K2rETI8frdVBIQIZyjJT7S2Pkge6ng0tHeOhoUpug4QQJ2LXvezXpLmqWjwKpj6bw",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -12.1816415,
                    "lng": 26.3524322
                },
                "viewport": {
                    "northeast": {
                        "lat": -12.1802954197085,
                        "lng": 26.3537818802915
                    },
                    "southwest": {
                        "lat": -12.1829933802915

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAPUcDoWNJkwPUFVJTdADqQDEt1napO2NVarVH84RpdOeudHUxlZQ_fsVohOTaInCHG42c6ETYb-4HQA5I3C1yvPxJmMJtPOiDc22W8aYHcx1qtxiGzVcUX4uBRpGzLZyduv40OHX80vVInHHNMyIJgI1caTztaRl-sF05-u2ITlAcuih5ZlJx4Rahe3lLLMXazhPVTIn8Ht11VICTKYdKsN4Qxtm47eZnwOxAOZfyb1mPODZoZIbrK-3N92T1g4gdIoLADT6ZWLeV3MzXU5IOqtQOTf92kaLh42q9vPtvvw2ZqCmJBjxQ_hU6KXgd6aIHanKiLdXFxRdI0JCEJfLlTVB3tMvhwyw3a3XfmrxUKqWkj_ewheX-cUaOGLjwJKzuxIQev0qJHNLEhDvCNBxDQYTYBoUEMz1tIZ53NcD93QfUV7_MkGvtHc",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -17.7920609,
                    "lng": 25.1561411
                },
                "viewport": {
                    "northeast": {
                        "lat": -17.7908848697085,
                        "lng": 25.1576967302915
                    },
                    "southwest": {
                        "lat": -17.7935828302915

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAALmA_vgLosxQBX-hr_c2n-sEGrU9sSC5XvXvG7XPA1X5ZOqincqB9mydixphZcfgZzwJfGYdQQksLNGgze4Y9D32MZHf2qfVKviKevBTzeQ2DwqAYUgjUNcwr7m6yaghLFTfZUC6RFY0xkcWHjLYulGz_adWE3YIy9DPm04ndghlFhY84hdMQKAWrjxM0AUjLGcJjX2KZMBuhjVxUoWD44wzue7jEysGmqfs1dxmTsWdH0oaYWmjiV1-G4zbvGufF7wQ-OPyJ_Aq383vjUEFu8jNZOWHKmeBB6Wami0vc9gYGo_T_TadjH0TjmvN265NMPWesLXmwMpyefnKbuU0obmPBFbsPAfktkW6zBs9zxfeCv9dYD0EbZzZGgnYu4zINBIQcKdPU0gVJ7XHvzTe05xTUBoUKX3-KuL4luy0gGueETaMbNpT7sY",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -26.79522029999999,
                    "lng": -60.4452856
                },
                "viewport": {
                    "northeast": {
                        "lat": -26.7939034197085,
                        "lng": -60.44384381970849
                    },
                    "southwest": {
                        "lat": -26.796

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAK2x8F5nKe6f8nn1iRb7fpO0q8_K7cuAD1e1zvyIgJnslqdVD4WaWNkY2FDMmh1HswbZEP6OfAwWgkPNd3gsP6Ul_czoyUZAl41cfFfDqP3KGC-DnH3k3ao0ZlbEzIORlo8vIhwrVFzZ9e5I1QlLeDTWaJW9BpjnRHdZD79bbJ83hLMqdlAemFNeOzYRt5AyUEIDN-OOjVz190mzSuW4F0hYzNVmktHEGy4UHHHkE_U3RZnYJzUsuFAChK9b-__Lmsyuuh2Nju8dhJ1skE7HKbv3TQXSbJvjU8bjpREpSCi3jg02TgjYvMQS8OcGSXr02g7tL8_LVuT2W8hcTkRsxmJCoEMaT3ypeud4Iw7iDqtDoeK1uYwFIu_yxeIs4dVTehIQ_9wawyjYeOKLAzh9h697NRoUT-R0clyRmFQWMM5pXY8FsvQ10VE",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.9851587,
                    "lng": 27.9235698
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.9837851197085,
                        "lng": 27.9248452302915
                    },
                    "southwest": {
                        "lat": -32.9864830802914

{
    "html_attributions": [],
    "next_page_token": "CqQCHAEAAEJ62dyJYqHop4O7e3AfbYr3YiH4bJV0irhdCOoRTCT2B0J4skWPXfXlAvFDBipguiWppPN80YICeXfcrIHcxA52DAEQbxPT0zjVGMEtmoUUcud06x1FPsSWU1LkLycVipv1NFcTqzVah6Ha3dTxxQOd7X7q7PTEGDPOPIZx5wMyQEof6KIaX1_vNCBVpcc3M_Uc0HDKrGWSClZlSVsIkaQS29h6p-m6MVEof4tlXPm_3NTZk2NnnnexA23d5ho1YbR9UxdEAyTrAR7e0_CxZv7YrZ18hzfWiEe14IeSKqoZJHNO24OGWMbAcgnUpOaxNOMMkd_k7bgQm5xyraD3zRKXcQ39Ln7ngxBe3fRauqfaSONnX4iKlWgZTJu5m3PmDRIQlWmkNQrPyoCsM209U-r6JhoUflkGGrtOr8KyaudC-8KNKm7AGbk",
    "results": [
        {
            "business_status": "OPERATIONAL",
            "geometry": {
                "location": {
                    "lat": -32.258535,
                    "lng": 24.536006
                },
                "viewport": {
                    "northeast": {
                        "lat": -32.2571630197085,
                        "lng": 24.5374605302915
                    },
                    "southwest": {
                        "lat": -32.2598609802915,


{
    "html_attributions": [],
    "results": [],
    "status": "ZERO_RESULTS"
}
Missing field/result... skipping.


In [37]:
hotel_df

,City,Lat,Lng,Max_Temp,Humidity,Cloudiness,Wind_Speed,Country,Date,Hotel_Name
83,Kabompo,-13.59,24.20,65.16,35,0,5.12,ZM,1595953867,Faroja City Iyindeko Cas-jay
121,Karratha,-20.74,116.85,67.87,34,0,5.44,AU,1595953861,ibis Styles Karratha
193,Solwezi,-12.17,26.39,62.26,39,0,4.52,ZM,1595953882,Country Lodge Solwezi
234,Kasane,-17.82,25.15,73.40,25,0,8.05,BW,1595953888,The Old House
259,Presidencia Roque Sáenz Peña,-26.79,-60.44,60.26,33,22,10.00,AR,1595953891,Hotel Aconcagua
262,East London,-33.02,27.91,67.78,50,0,1.79,ZA,1595953892,Tu Casa
442,Graaff-Reinet,-32.25,24.53,66.99,27,0,1.01,ZA,1595953916,Rietjiesbos B&B
521,Shubarkuduk,49.14,56.49,72.01,40,10,8.25,KZ,1595953926,


In [38]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel_Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# display figure
fig

Figure(layout=FigureLayout(height='420px'))